### 패키지 설치

In [2]:
!pip install torch==1.8.1

### 사용 데이터를 grownet 코드에 맞게 가공

#### 데이터 load

In [3]:
import pandas as pd
data = pd.read_csv('housing.csv')
data.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


#### 범주형 변수를 수치형으로 변환

In [4]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le = le.fit(data['ocean_proximity'])  
data['ocean_proximity'] = le.transform(data['ocean_proximity'])
data.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,3
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,3
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,3
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,3
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,3


In [5]:
data['ocean_proximity'].value_counts()

0    9136
1    6551
4    2658
3    2290
2       5
Name: ocean_proximity, dtype: int64

#### na 처리

In [6]:
data.isnull().sum()   # na 확인

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

In [7]:
data = data.dropna(axis=0)   # na 값이 포함된 행은 제거

In [8]:
data.isnull().sum()   # na 값이 제거되었는지 확인

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
median_house_value    0
ocean_proximity       0
dtype: int64

#### train, test set split 및 데이터 저장

In [10]:
# train, test split
from sklearn.model_selection import train_test_split
X = data.drop('median_house_value', axis=1)
y = data['median_house_value']
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=777)

# save files
import numpy as np
np.savez('train', labels = train_y, features = train_x )
np.savez('test', labels = test_y, features = test_x)

# 파일을 로드하여 잘 저장되었는지 확인
t = np.load('train.npz')
t['features']

array([[-1.1973e+02,  3.6750e+01,  3.9000e+01, ...,  3.0300e+02,
         3.1719e+00,  1.0000e+00],
       [-1.1815e+02,  3.4150e+01,  4.9000e+01, ...,  1.7200e+02,
         2.3654e+00,  0.0000e+00],
       [-1.2251e+02,  3.7530e+01,  1.7000e+01, ...,  2.4100e+02,
         7.2929e+00,  4.0000e+00],
       ...,
       [-1.2100e+02,  3.7610e+01,  3.6000e+01, ...,  5.5000e+02,
         2.2730e+00,  1.0000e+00],
       [-1.2248e+02,  3.7730e+01,  3.8000e+01, ...,  7.7800e+02,
         3.1359e+00,  4.0000e+00],
       [-1.1734e+02,  3.3150e+01,  1.7000e+01, ...,  1.0620e+03,
         3.3536e+00,  4.0000e+00]])

#### 설명변수 dimension 확인

In [12]:
train_x.shape[1]

9

In [15]:
!python -u main_reg_cv.py \
    --feat_d 9 \
    --hidden_d 32 \
    --boost_rate 1 \
    --lr 0.005 \
    --L2 .0e-3 \
    --num_nets 40 \
    --data $california \
    --tr train.npz \
    --te test.npz \
    --batch_size 2048 \
    --epochs_per_stage 1 \
    --correct_epoch 1 \
    --normalization True \
    --cv True \
    --out_f california_model.pth \
    --filename california_pred.csv

9
9
Creating Validation set! 

#Train: 15528, #Val: 818 #Test: 4087
$california training and test datasets are loaded!
Stage - 0, training time:  4.3 sec, model MSE loss:  115844.56866, Ensemble Net MSE Loss:  0.00000
Stage: 0  RMSE@Tr: 115844.48548, RMSE@Val: 114705.60133, RMSE@Te: 114011.85510
Stage - 1, training time:  9.7 sec, model MSE loss:  115843.33487, Ensemble Net MSE Loss:  115841.60176
Stage: 1  RMSE@Tr: 115841.55001, RMSE@Val: 114702.82940, RMSE@Te: 114009.03038
Stage - 2, training time:  8.9 sec, model MSE loss:  115839.64761, Ensemble Net MSE Loss:  115837.18180
Stage: 2  RMSE@Tr: 115834.30217, RMSE@Val: 114696.08007, RMSE@Te: 114002.04686
Stage - 3, training time:  10.0 sec, model MSE loss:  115834.67814, Ensemble Net MSE Loss:  115831.74741
Stage: 3  RMSE@Tr: 115827.17723, RMSE@Val: 114688.99107, RMSE@Te: 113994.93266
Stage - 4, training time:  10.6 sec, model MSE loss:  115828.17087, Ensemble Net MSE Loss:  115824.54181
Stage: 4  RMSE@Tr: 115817.01357, RMSE@Val: 11467